In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\ML_Projects\\Violence-Reporter'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    dataset_dir: Path
    classes_list: list
    params_image_height: int
    params_image_width: int
    params_sequence_length: int

In [5]:
from violenceReporter.constants import *
from violenceReporter.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            dataset_dir= config.dataset_dir,
            classes_list= config.classes_list,
            params_image_height= self.params.IMAGE_HEIGHT,
            params_image_width= self.params.IMAGE_WIDTH,
            params_sequence_length= self.params.SEQUENCE_LENGTH
        )

        return data_transformation_config

In [7]:
import cv2
import os
import numpy as np

In [8]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config=config

    def frames_extraction(self,video_path)->list:
        '''
        Extracts frames and normalize the frames from video
        '''
        frames_list = []
        video_reader = cv2.VideoCapture(video_path)
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count/self.config.params_sequence_length), 1)

        for frame_counter in range(self.config.params_sequence_length):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            success, frame = video_reader.read()

            if not success:
                break

            resized_frame = cv2.resize(frame, (self.config.params_image_height, self.config.params_image_width))
            normalized_frame = resized_frame / 255
            frames_list.append(normalized_frame)

        video_reader.release()
        return frames_list
    

    def create_dataset(self):
        ''' 
        Extract video features and labels and convert in numpy array
        '''
        features = []
        labels = []
        video_files_paths = []

        for class_index, class_name in enumerate(self.config.classes_list):
            print(f'Extracting Data of Class: {class_name}')
            files_list = os.listdir(os.path.join(self.config.dataset_dir, class_name))

            for file_name in files_list:
                video_file_path = os.path.join(self.config.dataset_dir, class_name, file_name)
                frames = self.frames_extraction(video_file_path)

                if len(frames) == self.config.params_sequence_length:
                    features.append(frames)
                    labels.append(class_index)
                    video_files_paths.append(video_file_path)

        features = np.asarray(features)
        labels = np.array(labels)

        return features, labels, video_files_paths
    
    def transform_dataset(self):
        '''
        Save transformed dataset to specified location
        '''
        features, labels, video_files_paths = self.create_dataset()
        np.save(f"{self.config.root_dir}/features.npy", features)
        np.save(f"{self.config.root_dir}/labels.npy", labels)
        np.save(f"{self.config.root_dir}/video_files_paths.npy", video_files_paths)
        
    
    

In [9]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.transform_dataset()
except Exception as e:
    raise e

[2024-03-31 00:29:26,237: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-31 00:29:26,256: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-31 00:29:26,260: INFO: common: created directory at: artifacts]
[2024-03-31 00:29:26,264: INFO: common: created directory at: artifacts/data_transformation]


Extracting Data of Class: NonViolence
Extracting Data of Class: Violence
